<a href="https://colab.research.google.com/github/mulyanoff/Adaptive-yarn-gulp-webpack-scss/blob/main/Laba_surprise!!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader

from collections import defaultdict
from operator import itemgetter
import heapq

import os
import csv

# Загружаем Датасет из Movielab
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
!ls

--2022-12-16 06:11:16--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.53MB/s    in 0.6s    

2022-12-16 06:11:17 (1.53 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
ml-latest-small  ml-latest-small.zip  sample_data


In [ ]:
# Загружаем рейтинги фильмов и возвращаем набор данных..
def load_dataset():
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    ratings_dataset = Dataset.load_from_file('ml-latest-small/ratings.csv', reader=reader)

    #Находим название фильма с его идентификатором Movielens в качестве ключа
    movieID_to_name = {}
    with open('ml-latest-small/movies.csv', newline='', encoding='ISO-8859-1') as csvfile:
            movie_reader = csv.reader(csvfile)
            next(movie_reader)
            for row in movie_reader:
                movieID = int(row[0])
                movie_name = row[1]
                movieID_to_name[movieID] = movie_name
    # Везвращаем набор данных и словарь поиска в массиве данных
    return (ratings_dataset, movieID_to_name)

dataset, movieID_to_name = load_dataset()

# Создаем полный обучающий набор Surprise из набора данных
trainset = dataset.build_full_trainset()

In [ ]:
similarity_matrix = KNNBasic(sim_options={
        'name': 'cosine',
        'user_based': False
        })\
        .fit(trainset)\
        .compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Выбираем случайный идентификатор пользователя, он должен быть числовой строкой.

test_subject = '70'

# Получаем 20 лучших элементов, оцененных пользователями
k = 20

In [ ]:
# При использовании Surprise есть RAW и INNER ID.
# Необработанные идентификаторы строки или числа
# Преобразовываем RAW в INNER
test_subject_iid = trainset.to_inner_uid(test_subject)

# Получаем лучшие элемменты после оценки
test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbors = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [ ]:
# Создаем новую запись с ключом к Датасету
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue

In [ ]:
# Создаем утилиту для дальнейшего использования
def getMovieName(movieID):
  if int(movieID) in movieID_to_name:
    return movieID_to_name[int(movieID)]
  else:
      return ""

In [ ]:
# Создаем словарь фильмов, которые смотрел пользователь
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  watched[itemID] = 1


# Добавляем элементы в список рекомендаций пользователя
# Если они похожи на их любимые фильмы,
# И еще не смотрели
recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in watched:
    recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
    position += 1
    if (position > 10): break # Нам нужно только топ 10

for rec in recommendations:
  print("Movie: ", rec)

Movie:  Bride & Prejudice (2004)
Movie:  Quo Vadis (1951)
Movie:  Safety Last! (1923)
Movie:  Infamous (2006)
Movie:  Oliver Twist (1948)
Movie:  Return of the Secaucus 7 (1980)
Movie:  Dark Days (2000)
Movie:  Charlotte's Web (2006)
Movie:  Only Angels Have Wings (1939)
Movie:  Little Women (1949)
Movie:  Mr. Deeds Goes to Town (1936)
